In [1]:
!pip install transformers evaluate sacrebleu datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import evaluate
from datasets import load_dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
metric = evaluate.load("sacrebleu")

In [5]:
dataset_en = load_dataset("facebook/flores", "eng_Latn", trust_remote_code=True)
dataset_kk = load_dataset("facebook/flores", "kaz_Cyrl", trust_remote_code=True)

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

In [6]:
devtest_dataset_en = dataset_en['devtest']
flores_en = []

for item in devtest_dataset_en:
    if not item['has_hyperlink']:
        flores_en.append(item['sentence'])

flores_en

['"We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.',
 'Dr. Ehud Ur, professor of medicine at Dalhousie University in Halifax, Nova Scotia and chair of the clinical and scientific division of the Canadian Diabetes Association cautioned that the research is still in its early days.',
 'Like some other experts, he is skeptical about whether diabetes can be cured, noting that these findings have no relevance to people who already have Type 1 diabetes.',
 'On Monday, Sara Danius, permanent secretary of the Nobel Committee for Literature at the Swedish Academy, publicly announced during a radio program on Sveriges Radio in Sweden the committee, unable to reach Bob Dylan directly about winning the 2016 Nobel Prize in Literature, had abandoned its efforts to reach him.',
 'Danius said, "Right now we are doing nothing. I have called and sent emails to his closest collaborator and received very friendly replies. For now, that is certainly enough."',
 "Prev

In [7]:
devtest_dataset_kk = dataset_kk['devtest']
flores_kk = []

for item in devtest_dataset_kk:
    if not item['has_hyperlink']:
        flores_kk.append(item['sentence'])

flores_kk

['«Қазір бізде диабетпен ауырған, бірақ қазір диабеті жоқ 4 айлық тышқандар бар», - деп қосты ол.',
 'Жаңа Шотландия провинциясының Галифакс қаласындағы Далхоузи университетінің медицина профессоры және Канадалық диабеттер қауымдастығының клиникалық және ғылыми бөлімі басшысы доктор Эхуд Ур зерттеудің алғашқы кезеңі жалғасып жатқандығын ескертті.',
 'Кейбір басқа сарапшылар сияқты, ол диабетті емдеуге болатынына күмәнмен қарайды, бұл ашулардың 1-түрге жататын диабетімен ауыратын адамдарға қатысы жоқ екенін ескертті.',
 'Дүйсенбіде Швеция академиясы жанындағы әдебиет саласы бойынша Нобель комитетінің тұрақты хатшысы Сара Даниус Швециядағы Сверигес радиосындағы радиобағдарлама барысында 2016 жылғы әдебиет саласындағы Нобель сыйлығын жеңіп алу туралы Боб Диланға хабарлай алмағанын жариялап, бұл әрекеттерін тоқтатқанын айтты.',
 'Даниус былай деді: «Қазіргі таңда біз дым да істеп жатқан жоқпыз. Мен оның ең жақын серіктесіне телефон да соқтым, имейл де жібердім және өте жылы жауап алдым. Бү

In [ ]:
predictions_kk = []
references_kk = []

for i, sentence in enumerate(flores_en):
    inputs = tokenizer(sentence, return_tensors="pt")

    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["kaz_Cyrl"], max_length=400
    )
    predictions_kk.append(tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0])
    references_kk.append([flores_kk[i]])


In [ ]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

bleu.corpus_score(predictions_kk, references_kk)

In [ ]:
predictions_en = []
references_en = []

for i, sentence in enumerate(flores_kk):
    inputs = tokenizer(sentence, return_tensors="pt")

    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=400
    )
    predictions_en.append(tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0])
    references_en.append([flores_en[i]])


In [ ]:
bleu.corpus_score(predictions_en, references_en)